In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error

from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_json('Cell_Phones_and_Accessories.json')
df.head(5)

,Rating,Rev_verify,Review_Date,IC,Prod_meta,Reviewer_Name,Review,Rev_summ,Review_timestamp,Useful,Prod_img
0,5,True,"09 1, 2015",B009XD5TPQ,None,Sunny Zoeller,Bought it for my husband. He's very happy with it,He's very happy with,1441065600,None,None
1,5,True,"01 9, 2016",B016MF3P3K,None,Denise Lesley,Great screen protector. Doesn't even seem as ...,Five Stars,1452297600,None,None
2,5,True,"04 21, 2013",B008DC8N5G,None,Emir,Saved me lots of money! it's not gorilla glass...,As long as you know how to put it on!,1366502400,None,None
3,3,True,"02 27, 2013",B0089CH3TM,{'Color:': ' Green'},Alyse,"The material and fit is very nice, but the col...",Good case overall,1361923200,3,None
4,4,True,"12 19, 2013",B00AKZWGAC,None,TechGuy,This last me about 3 days till i have to charg...,Awesome Battery,1387411200,None,None


In [3]:
df.drop(['Rating','Rev_summ','Rev_verify','Review_Date','Prod_meta','Reviewer_Name','Review_timestamp','Useful','Prod_img'],axis=1,inplace=True)

In [4]:
df.isnull().sum()

IC          0
Review    530
dtype: int64

In [5]:
df['Review'].fillna(df['Review'].mode().values[0],inplace=True)

In [6]:
df.isnull().sum()

IC        0
Review    0
dtype: int64

In [7]:
df['Review'] = df['Review'].apply(lambda x : x+' ')

In [8]:
df

,IC,Review
0,B009XD5TPQ,Bought it for my husband. He's very happy with...
1,B016MF3P3K,Great screen protector. Doesn't even seem as ...
2,B008DC8N5G,Saved me lots of money! it's not gorilla glass...
3,B0089CH3TM,"The material and fit is very nice, but the col..."
4,B00AKZWGAC,This last me about 3 days till i have to charg...
...,...,...
760445,B00C3V9M8A,Very good
760446,B0178BYS24,My name is Cynthia Beard and I believe that th...
760447,B009KY47CE,This iphone case is very durable and long last...
760448,B00X60AYDY,great


In [9]:
df=df.groupby(by = ['IC']).sum()

In [10]:
df

,Review
IC,
7508492919,It is such a good case for a low price. I have...
7532385086,I was very excited when I first got this case....
7887421268,Good item Received this in about a weeks time....
8199900164,thank you. Works some of the time. The charge...
8288853439,Great ! So far.\nI had this charger before and...
...,...
B01HJBS5C2,"Strong, protective but slim and light. I am al..."
B01HJC7N4C,my sons likes it. For the price it isn't too b...
B01HJCN1UC,great thanks good cable. just be sure to know ...


In [11]:
for columns in df.columns:
    df[columns] = df[columns].str.lower()

In [12]:
df['Review'] = df['Review'].str.replace(r'[^\w\d\s]', ' ')
df['Review'] = df['Review'].str.replace("\n", ' ')
# Replace whitespace between terms with a single space
df['Review'] = df['Review'].str.replace(r'\s+', ' ')
# Remove leading and trailing whitespace
df['Review'] =df['Review'].str.replace(r'^\s+|\s+?$', '')

In [13]:
#move stopwords
import string
import nltk
from nltk.corpus import  stopwords

stop_words = set(stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure'])

df['Review'] = df['Review'].apply(lambda x: ' '.join(
    term for term in x.split() if term not in stop_words))

In [14]:
from collections import Counter
from nltk.corpus import stopwords

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.stem import WordNetLemmatizer

from nltk.tokenize import word_tokenize

import nltk
nltk.download('vader_lexicon')
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package vader_lexicon to C:\Users\RIA
[nltk_data]     MAITRA\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [15]:
df['Review']=df.apply(lambda row: nltk.word_tokenize(row['Review']),axis=1)

In [16]:
!pip install gensim
import gensim

# Converts into tokens (Alternative to word_tokenize)
from gensim.utils import simple_preprocess

from gensim.parsing.preprocessing import STOPWORDS

from nltk.stem import WordNetLemmatizer, SnowballStemmer
#from nltk.stem import *
import numpy as np
np.random.seed(400)

In [17]:
from gensim import corpora
dictionary= corpora.Dictionary(df['Review'])

In [18]:
print(dictionary)

Dictionary(107984 unique tokens: ['1', '3', '3d', '4s', '5c']...)


In [19]:
df

,Review
IC,
7508492919,"[good, case, low, price, right, never, problem..."
7532385086,"[excited, first, got, case, loved, color, feel..."
7887421268,"[good, item, received, weeks, time, nice, actu..."
8199900164,"[thank, works, time, charger, stopped, working..."
8288853439,"[great, far, charger, seem, last, pretty, good..."
...,...
B01HJBS5C2,"[strong, protective, slim, light, already, ord..."
B01HJC7N4C,"[sons, likes, price, bad, recently, received, ..."
B01HJCN1UC,"[great, thanks, good, cable, sure, know, lengt..."


In [20]:
dictionary.filter_extremes(no_below=15,no_above=0.1,keep_n=200000)

In [21]:
print(dictionary)

Dictionary(15754 unique tokens: ['3d', '4s', '5c', 'amazed', 'ans']...)


In [22]:
df['bow_corpus'] = df['Review'].map(dictionary.doc2bow)

In [23]:
lda_model =  gensim.models.LdaMulticore(df['bow_corpus'], 
                                   num_topics = 2, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [37]:
for idx, topic in lda_model.print_topics():
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.007*"watch" + 0.005*"band" + 0.004*"bubbles" + 0.004*"cards" + 0.004*"tempered" + 0.004*"s7" + 0.004*"wallet" + 0.003*"leather" + 0.003*"belt" + 0.003*"holster"


Topic: 1 
Words: 0.007*"cables" + 0.006*"mount" + 0.005*"cord" + 0.005*"charged" + 0.004*"anker" + 0.004*"images" + 0.004*"unit" + 0.003*"chargers" + 0.003*"bluetooth" + 0.003*"android"




In [26]:
for index, score in sorted(lda_model[df['bow_corpus'][0]]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 30)))

Score: 0.9965626001358032	 Topic: 0.007*"watch" + 0.005*"band" + 0.004*"bubbles" + 0.004*"cards" + 0.004*"tempered" + 0.004*"s7" + 0.004*"wallet" + 0.003*"leather" + 0.003*"belt" + 0.003*"holster" + 0.003*"spigen" + 0.003*"gold" + 0.003*"apply" + 0.003*"installation" + 0.003*"6s" + 0.003*"water" + 0.002*"corners" + 0.002*"magnet" + 0.002*"kickstand" + 0.002*"credit" + 0.002*"slippery" + 0.002*"air" + 0.002*"bumper" + 0.002*"discounted" + 0.002*"film" + 0.002*"please" + 0.002*"tpu" + 0.002*"magnetic" + 0.002*"offers" + 0.002*"metal"


In [27]:
for index, score in sorted(lda_model[df['bow_corpus'][1]]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 30)))

Score: 0.968845009803772	 Topic: 0.007*"watch" + 0.005*"band" + 0.004*"bubbles" + 0.004*"cards" + 0.004*"tempered" + 0.004*"s7" + 0.004*"wallet" + 0.003*"leather" + 0.003*"belt" + 0.003*"holster" + 0.003*"spigen" + 0.003*"gold" + 0.003*"apply" + 0.003*"installation" + 0.003*"6s" + 0.003*"water" + 0.002*"corners" + 0.002*"magnet" + 0.002*"kickstand" + 0.002*"credit" + 0.002*"slippery" + 0.002*"air" + 0.002*"bumper" + 0.002*"discounted" + 0.002*"film" + 0.002*"please" + 0.002*"tpu" + 0.002*"magnetic" + 0.002*"offers" + 0.002*"metal"
Score: 0.031154992058873177	 Topic: 0.007*"cables" + 0.006*"mount" + 0.005*"cord" + 0.005*"charged" + 0.004*"anker" + 0.004*"images" + 0.004*"unit" + 0.003*"chargers" + 0.003*"bluetooth" + 0.003*"android" + 0.003*"wireless" + 0.003*"micro" + 0.003*"sound" + 0.003*"0" + 0.003*"ipad" + 0.003*"tablet" + 0.003*"pack" + 0.003*"wall" + 0.003*"class" + 0.003*"apps" + 0.002*"data" + 0.002*"plugged" + 0.002*"nexus" + 0.002*"c" + 0.002*"adapter" + 0.002*"bank" + 0.002*

In [36]:
 for index, score in sorted(lda_model[df['bow_corpus']]):
        print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 30)))

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices